# LangSmithを使った評価
LangSmithの基本的なトレーシングの方法と、feedbackやevaluationを使った評価を行う。

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## LangSmith関連の環境変数

In [2]:
import os

print(f"""Env Variables
LANGCHAIN_TRACING_V2: {os.environ["LANGCHAIN_TRACING_V2"]}
LANGCHAIN_PROJECT: {os.environ["LANGCHAIN_PROJECT"]}
LANGCHAIN_ENDPOINT: {os.environ["LANGCHAIN_ENDPOINT"]}
""")

Env Variables
LANGCHAIN_TRACING_V2: true
LANGCHAIN_PROJECT: machine-learning-workshop
LANGCHAIN_ENDPOINT: https://api.smith.langchain.com



## LangSmithのトレースの基本
traceableデコレータを使用することで、任意の関数の引数と返り値をLangSmithで確認できるようになる。

In [3]:
from IPython.display import display, Markdown
from langsmith import traceable
import openai

openai_client = openai.Client()


@traceable
def format_prompt(question):
    return [
        {
            "role": "system",
            "content": "あなたはBigQueryのエキスパートです. 出したいデータのクエリを作成してください. 出力はクエリのみで他の情報は不要です.",
        },
        {"role": "user", "content": f"{question}"},
    ]


@traceable(run_type="llm")
def invoke_llm(messages):
    return openai_client.chat.completions.create(
        messages=messages, model="gpt-4o", temperature=0
    )


@traceable
def parse_output(response):
    return response.choices[0].message.content


@traceable
def run_pipeline():
    messages = format_prompt("ウェブサイトの回遊率")
    response = invoke_llm(messages)
    return parse_output(response)


display(Markdown(run_pipeline()))

```sql
SELECT
  user_id,
  COUNT(DISTINCT session_id) AS total_sessions,
  COUNT(DISTINCT page_id) AS total_pages,
  COUNT(DISTINCT page_id) / COUNT(DISTINCT session_id) AS page_per_session
FROM
  `your_project.your_dataset.your_table`
GROUP BY
  user_id
```

openaiとのやり取りの可観測にするラッパー `wrap_openai` を使うと、モデルの情報などを簡単に取得可能になる。

In [4]:
from langsmith.wrappers import wrap_openai

wrap_openai_client = wrap_openai(openai.Client())


@traceable(name="run_pipeline with wrap_openai")
def run_pipeline_with_wrap_llm():
    messages = format_prompt("ウェブサイトの回遊率")
    response = wrap_openai_client.chat.completions.create(
        messages=messages, model="gpt-4o", temperature=0
    )
    return parse_output(response)


display(Markdown(run_pipeline_with_wrap_llm()))

```sql
SELECT
  (SUM(CASE WHEN pageviews > 1 THEN 1 ELSE 0 END) / COUNT(*)) AS bounce_rate
FROM
  `your_dataset.your_table`
WHERE
  session_id IS NOT NULL;
```

LCEL (LangChain Expression Language) を使えば、LangSmithでの観測が楽にできる。

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたはBigQueryのエキスパートです. 出したいデータのクエリを作成してください. 出力はクエリのみで他の情報は不要です.",
        ),
        ("human", "{question}"),
    ]
)
output_parser = StrOutputParser()

chain = prompt | model | output_parser
display(Markdown(chain.invoke("ウェブサイトの回遊率")))

以下のクエリは、ウェブサイトの全セッション中でページビューションが2回以上あるセッションの割合（回遊率）を計算します。ここでは、`website_log`という仮想のテーブルを使用し、各セッション（`session_id`）のページビュー数（`page_views`）をカウントします。

```sql
SELECT 
  (COUNTIF(page_views_per_session >= 2) / COUNT(*)) AS bounce_rate
FROM (
  SELECT 
    session_id, 
    COUNT(*) AS page_views_per_session
  FROM 
    `your_project.your_dataset.website_log`
  GROUP BY 
    session_id
)
```

このクエリは、各セッションに対するページビュー数を集計し、その中で2回以上のページビューがあるセッションの割合を計算します。`COUNTIF`関数は、指定した条件に一致する行をカウントします。この場合、それはページビュー数が2以上の行です。そして、`COUNT(*)`は全体のセッション数をカウントします。これらの比率を取ることで、ウェブサイトの回遊率を計算しています。

## FeedbackとEvaluationを使った評価


In [6]:
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt


def mm(graph):
    graphbytes = graph.encode("ascii")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

In [7]:
mm("""
graph TD
  A[LangSmith] --> B[User and Product Team Feedback]
  A --> C[Prepared Dataset Experiments]

  subgraph B[Feedback]
    B1[Annotate Runs] --> B2[Save Feedback]
  end

  subgraph C[Evaluation]
    C1[Input and Output] --> C2[Run Experiments]
  end

  classDef main fill:#f9f,stroke:#333,stroke-width:2px;
  classDef sub fill:#bbf,stroke:#333,stroke-width:2px;
  classDef detail fill:#fb3,stroke:#333,stroke-width:2px;

  class A,B,C main;
  class B1,B2,C1,C2 detail;   
   """)

### Feedback
- Traceされた実行の中に含まれるRunに、自分で定義したTagやKeyをAnnotate
  - trace_id1つに対して複数のrun_idが含まれる構造
  - 最初のrun_idはtrace_idと同一
- API経由のfeedbackではKey, 手動のfeedbackではTagでAnnotateする仕組みとなっているが、TagもKeyとして保存されている
"- ただし、API経由のfeedbackはrecordが追加・上書きできるのに対して、手動のfeedbackは上書きのみという違いがある
"- 数値データで同じキーのものは集計されて表示される
- LLMアプリを使っているユーザからのフィードバックは、基本的にAPI経由の登録となる


In [8]:
from langchain_openai import ChatOpenAI
from langchain import hub


llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()
tools = [search]

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'how can LangSmith help with testing'}`




LangSmith上で手動でFeedbackを付与することができる

#### Runの情報を取得
LangSmithのAPIを使ってFeedbackを作成するためには、run idを取得する必要がある。

In [ ]:
from langchain.callbacks import tracing_v2_enabled

with tracing_v2_enabled() as cb:
    agent_executor.invoke({"input": "how can langsmith help with testing?"})

In [ ]:
# RunのURLを取得
cb.get_run_url()

In [ ]:
# Run IDを取得
run_id = cb.latest_run.id
run_id

#### Feedbackの作成
run_idを指定してFeedbackを追加することができる。
keyは自分で任意の値を指定することが可能。

In [ ]:
from langsmith import Client

ls_client = Client()

score_feedback1 = ls_client.create_feedback(
    run_id=cb.latest_run.id, key="test-score", score=1
)
score_feedback1

同じキーを指定すると別のfeedbackとして保存される

In [ ]:
score_feedback2 = ls_client.create_feedback(
    run_id=cb.latest_run.id, key="test-score", score=100
)
score_feedback2

feedback_idを指定して、上書きすることも可能。

In [ ]:
update_score_feedback1 = ls_client.create_feedback(
    run_id=cb.latest_run.id, feedback_id=score_feedback1.id, key="test-score", score=500
)
update_score_feedback1

In [ ]:
comment_feedback = ls_client.create_feedback(
    run_id=cb.latest_run.id, key="test-comment", comment="test comment"
)
comment_feedback

#### Feedbackの削除
feedback_idを指定して、作成したfeedbackを削除することも可能。

In [ ]:
# 作成したfeedbackを削除
ls_client.delete_feedback(score_feedback1.id)
ls_client.delete_feedback(comment_feedback.id)

In [ ]:
# list_feedbackでfeedbackをすべて取得し、削除
for feedback in ls_client.list_feedback(run_ids=[cb.latest_run.id]):
    ls_client.delete_feedback(feedback.id)

### Evaluation
- DatasetにあらかじめInputとOutputの組み合わせからなるExampleを保存
- ExampleのInputを使ってLLMを実行し、出てきたOutputを保存されているOutputを使って評価
- 評価の方法は、LangSmithがあらかじめ用意しているものか、カスタムで作成することができる
- 評価結果は、key (評価指標の名前), score (評価結果), commentとして残すことが可能


In [ ]:
mm("""
graph TD
  subgraph EvaluationPipeline[Evaluation Pipeline]
    B[Datasets] --> C[Examples]
    C --> D[Inputs]
    C --> E[Expected Outputs]
    D --> F[LLM]
    F --> G[Run Outputs]
    E --> H[Evaluators]
    G --> H
    H --> I[Evaluation Result]

    subgraph I[Evaluation Result]
      direction LR
      I1[Key: metric name] -.- I2[Score: metric value] -.- I3[Comment: reasoning]
      I3 -.- I1
    end
  end

  classDef main fill:#f9f,stroke:#333,stroke-width:2px;
  classDef sub fill:#bbf,stroke:#333,stroke-width:2px;
  classDef detail fill:#fb3,stroke:#333,stroke-width:2px;

  class B,C,D,E,F,G,H,I main;
  class I1,I2,I3 detail;   
   """)

#### DatasetとExampleを作成
Datasetを作成し、Exampleを保存。

In [ ]:
import textwrap

from langsmith import Client
from langsmith.schemas import Run, Example
from langsmith.evaluation import evaluate

ls_client = Client()  # LangSmithのクライアント

# 作成するデータセット
dataset_name = "SQL Samples"

# データセットがあれば削除
if ls_client.has_dataset(dataset_name=dataset_name):
    dataset = ls_client.delete_dataset(dataset_name=dataset_name)

dataset = ls_client.create_dataset(
    dataset_name, description="ML Workshop用のサンプルクエリ"
)

# データセットにexampleを保存
ls_client.create_examples(
    inputs=[
        {"question": "MAUを取得"},
        {"question": "新規ユーザ数の推移"},
    ],
    outputs=[
        {
            "query": textwrap.dedent("""
           SELECT
               COUNT(DISTINCT user_id) AS monthly_active_users
           FROM
               `your_dataset.user_activities`
           WHERE
               activity_date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH) AND CURRENT_DATE()
        """),
            "tables": ["user_activities"],
        },
        {
            "query": textwrap.dedent("""
            SELECT
                signup_date,
                COUNT(user_id) AS new_users
            FROM
                `your_dataset.user_activities`
            GROUP BY
                signup_date
            ORDER BY
            　　 signup_date
        """),
            "tables": ["user_activities"],
        },
    ],
    dataset_id=dataset.id,
)

In [ ]:
# datasetのURLを取得
dataset.url

In [ ]:
# datasetのexample数を取得
dataset.example_count

example_countが0となっているので、LangSmith Clientを使ってdatasetを読み直す

In [ ]:
dataset = ls_client.read_dataset(dataset_name=dataset_name)
dataset.example_count

In [ ]:
ls_client.create_examples(
    inputs=[
        {"question": "月ごとのCV数の推移"},
    ],
    outputs=[
        {
            "query": textwrap.dedent("""
           SELECT
               FORMAT_TIMESTAMP('%Y-%m', conv_date) AS conversion_month,
               COUNT(conv_id) AS conversions
           FROM
               `your_dataset.your_table`
           WHERE
               conv_date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 1 YEAR) AND CURRENT_DATE()
           GROUP BY
               conversion_month
           ORDER BY
               conversion_month
        """),
            "tables": ["user_activities"],
        },
    ],
    dataset_id=dataset.id,
)

exampleを増やしたことにより、datasetのバージョンも変更されている

In [ ]:
ls_client.read_dataset(dataset_name=dataset_name).example_count

In [ ]:
# datasetに保存されているexampleの一覧
for example in ls_client.list_examples(dataset_name=dataset_name):
    print(f"""
question: {example.inputs["question"]}
query: {example.outputs["query"]}
    """)

### Custom Evaluationを実行

In [ ]:
# inputsにexampleが1つずつ渡される
def predict(inputs: dict) -> dict:
    model = ChatOpenAI(model="gpt-4")
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "あなたはBigQueryのエキスパートです. 出したいデータのクエリを作成してください. 出力はクエリのみで他の情報は不要です.",
            ),
            ("human", "{question}, tableはuser_activitiesを使います."),
        ]
    )
    output_parser = StrOutputParser()
    llm = prompt | model | output_parser
    return {"output": llm.invoke(inputs)}


# Custom Evaluation
def must_have_user_activities(run: Run, example: Example) -> dict:
    prediction = run.outputs.get("output") or ""
    print(f"run id: {run.id}\n")
    required = example.outputs.get("tables") or []  # outputsのキー (tables) と合わせる
    print(required)
    print(prediction)
    score = all(
        phrase in prediction for phrase in required
    )  # scoreは自分で定義したものでよい
    return {
        "key": "must_have_user_activities",
        "score": score,
        "comment": "comment test",
    }  # key, score, commentを返す


experiment_results = evaluate(
    predict,
    data=dataset_name,  # The data to predict and grade over
    evaluators=[must_have_user_activities],  # The evaluators to score the results
    experiment_prefix="ml-workshop",  # A prefix for your experiment names to easily identify them
    metadata={
        "version": "1.0.0",
    },
)